In [1]:
%punch_spark_session -f

23/02/01 10:27:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
INFO:SparkMonitorKernel:Client Connected ('127.0.0.1', 34270)


### Generate data to save

In [2]:
%%punch_source --type generator --name df -o
messages:
  - app: app1
    visits: 20
  - app: app2
    visits: 50
  - app: app2
    visits: 0
  - app: app1
    visits: 10000
  - app: app3
    visits: 100

Data is available in df variable.
Execution time: 0:00:01.785079


In [3]:
df.head(2)

[Row(app='app1', visits=20), Row(app='app2', visits=50)]

### ElasticSearch

In [4]:
%%punch_sink --type elasticsearch -df df
index:
    type: constant
    value: index-pyspark

Data saved.
Execution time: 0:00:07.038279


#####  Fetch data from elasticsearch

In [5]:
%%punch_source --type elasticsearch --name out -o
index: index-pyspark

Data is available in out variable.
Execution time: 0:00:01.558305


In [6]:
print(out.printSchema())
out.show()

root
 |-- app: string (nullable = true)
 |-- visits: long (nullable = true)

None
+----+------+
| app|visits|
+----+------+
|app2|     0|
|app1| 10000|
|app3|   100|
|app1|    20|
|app2|    50|
|app2|     0|
|app1| 10000|
|app3|   100|
|app1|    20|
|app2|    50|
|app1|    20|
|app2|    50|
|app2|     0|
|app1| 10000|
|app3|   100|
|app2|     0|
|app1| 10000|
|app3|   100|
|app1|    20|
|app2|    50|
+----+------+
only showing top 20 rows



In [7]:
del out

### Kafka

##### Save to kafka

In [8]:
%%punch_sink --type kafka -df df
topic: topic-pyspark

Data saved.
Execution time: 0:00:00.784325


### Fetch data from kafka

In [9]:
%%punch_source --type kafka --name out -o
topic: topic-pyspark

Data is available in out variable.
Execution time: 0:00:00.055489


In [10]:
print(out.printSchema())
out.show()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

None


+----+--------------------+-------------+---------+------+--------------------+-------------+
| key|               value|        topic|partition|offset|           timestamp|timestampType|
+----+--------------------+-------------+---------+------+--------------------+-------------+
|null|[7B 22 61 70 70 2...|topic-pyspark|        0|     0|2023-02-01 09:38:...|            0|
|null|[7B 22 61 70 70 2...|topic-pyspark|        0|     1|2023-02-01 09:38:...|            0|
|null|[7B 22 61 70 70 2...|topic-pyspark|        0|     2|2023-02-01 09:38:...|            0|
|null|[7B 22 61 70 70 2...|topic-pyspark|        0|     3|2023-02-01 09:38:...|            0|
|null|[7B 22 61 70 70 2...|topic-pyspark|        0|     4|2023-02-01 09:38:...|            0|
|null|[7B 22 61 70 70 2...|topic-pyspark|        0|     5|2023-02-01 09:50:...|            0|
|null|[7B 22 61 70 70 2...|topic-pyspark|        0|     6|2023-02-01 09:50:...|            0|
|null|[7B 22 61 70 70 2...|topic-pyspark|        0|     7|20

In [11]:
del out

### File (s3)

### csv

##### Save to file in s3

In [36]:
%%punch_sink --type file -df df
options:
    header: True
format: csv
path: s3a://default/pyspark/csv
save_mode: Overwrite

23/02/01 10:32:21 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
Data saved.
Execution time: 0:00:00.226006


##### Recover data from s3 (file node)

In [37]:
%%punch_source --type file --name out -o 
format: csv
options:
  header: true
path: s3a://default/pyspark/csv

Data is available in out variable.
Execution time: 0:00:00.118294


In [38]:
print(out.printSchema())
out.show()

root
 |-- app: string (nullable = true)
 |-- visits: string (nullable = true)

None
+----+------+
| app|visits|
+----+------+
|app2|     0|
|app1| 10000|
|app3|   100|
|app1|    20|
|app2|    50|
+----+------+



### parquet

##### Save to file in s3

In [39]:
%%punch_sink --type file -df df
format: parquet
path: s3a://default/pyspark/parquet
save_mode: Overwrite

Data saved.
Execution time: 0:00:00.218731


##### Recover data from s3 (file node)

In [40]:
%%punch_source --type file --name out -o 
format: parquet
path: s3a://default/pyspark/parquet

Data is available in out variable.
Execution time: 0:00:00.088897


In [41]:
print(out.printSchema())
out.show()

root
 |-- app: string (nullable = true)
 |-- visits: integer (nullable = true)

None
+----+------+
| app|visits|
+----+------+
|app2|     0|
|app1| 10000|
|app3|   100|
|app1|    20|
|app2|    50|
+----+------+



### json

##### Save to file in s3

In [42]:
%%punch_sink --type file -df df
format: json
path: s3a://default/pyspark/json
save_mode: Overwrite

23/02/01 10:32:39 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
Data saved.
Execution time: 0:00:00.211318


##### Recover data from s3 (file node)

In [43]:
%%punch_source --type file --name out -o 
format: json
path: s3a://default/pyspark/json

Data is available in out variable.
Execution time: 0:00:00.086513


In [44]:
print(out.printSchema())
out.show()

root
 |-- app: string (nullable = true)
 |-- visits: long (nullable = true)

None
+----+------+
| app|visits|
+----+------+
|app2|     0|
|app1| 10000|
|app3|   100|
|app1|    20|
|app2|    50|
+----+------+

